In [24]:
import h5py
from dataloader.semantickitti import SemanticKITTI
import yaml
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False


In [44]:

color_map = {
  0: [0,0,0],
  1: [245, 150, 100],
  2: [245, 230, 100],
  3: [150, 60, 30],
  4: [180, 30, 80],
  5: [255, 0, 0],
  6: [30, 30, 255],
  7: [200, 40, 255],
  8: [90, 30, 150],
  9: [255, 0, 255],
  10: [255, 150, 255],
  11: [75, 0, 75],
  12: [75, 0, 175],
  13: [0, 200, 255],
  14: [50, 120, 255],
  15: [0, 175, 0],
  16: [0, 60, 135],
  17: [80, 240, 150],
  18: [150, 240, 255],
  19: [0, 0, 255]
}

# sample = dataset[0]

def save_sample(xyzr, label, path='tmp/test.txt'):
    rgb = []
    with open(path, 'w') as f:
        for (x, y, z, _), l in zip(xyzr, label):
            r, g, b = color_map[l.item()]
            rgb.append(color_map[l.item()])
            print(f"{x.item()} {y.item()} {z.item()} {r} {g} {b}", file=f)
    rgb = np.array(rgb)
    print(rgb.shape)
    render(xyzr[:, :3], rgb)
def render(xyz, rgb, idx):
    x=xyz[:,0]
    y=xyz[:,1]
    z=xyz[:,2]
    
    fig=plt.figure()
    ax = fig.add_subplot(projection = '3d')
    ax.scatter(x,y,z,c=rgb/255.0)
    ax.set_title('三维图')
    
    plt.savefig(f"visualize/{idx}.png")
 

In [46]:

hf = h5py.File("./inference/training_results.h5", "r")
config = yaml.safe_load(open('config/unc_filter.yaml', 'r'))
config['dataset'].update(yaml.safe_load(open('config/dataset/semantickitti.yaml', 'r')))
ds = SemanticKITTI(split='train', config=config['dataset'])
for i in tqdm(range(len(ds))):
    label_path = ds.label_paths[i]
    
    xyzr = ds.get_lidar(i)

    predicted_labels = hf[os.path.join(label_path, 'label')][()]
    uncertainty_scores = hf[os.path.join(label_path, 'unc')][()]
    
    save_sample(xyzr, predicted_labels, i)

  0%|          | 0/19130 [00:00<?, ?it/s]

  0%|          | 0/19130 [00:00<?, ?it/s]


ValueError: IPython won't let you open fd=0 by default as it is likely to crash IPython. If you know what you are doing, you can use builtins' open.